In [1]:
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Pretty display for notebooks
%matplotlib inline


In [2]:
#Parameters
data_directory = "/Fergus/Code/Data"
file1 = "/Airbnb/Dublin/summary_listings.csv"
file2 = "/Airbnb/Dublin/listings.csv"
file3 = "/Airbnb/Dublin/reviews.csv/reviews.csv"
name_of_dependent_variable = 'income'
skewed = ['capital-gain', 'capital-loss']

numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [3]:
#Open data file
data_summary = pd.read_csv(data_directory + file1)
data_extended = pd.read_csv(data_directory + file2)
data_reviews = pd.read_csv(data_directory + file3)

C:\Users\Fergus\.conda\envs\uci_env\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [5]:
data_extended.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,14842,https://www.airbnb.com/rooms/14842,20190122134607,2019-01-22,"Best location Portobello entire house, can sle...","Beautiful entire house, newly revamped, with s...","Gorgeous, newly revamped central Dublin city e...","Beautiful entire house, newly revamped, with s...",none,Portobello is an exceptional place to live - v...,NaN,"There is great public transport - buses, Luas,...",Guests get the whole house to themselves.,I love meeting guests and I give you extensive...,"This house is a charming space, just completel...",NaN,NaN,https://a0.muscache.com/im/pictures/106656330/...,NaN,58344,https://www.airbnb.com/users/show/58344,Hilary,2009-11-29,"Dublin, County Dublin, Ireland",An easy-going and friendly Irish woman. Works ...,within an hour,100%,NaN,t,https://a0.muscache.com/im/users/58344/profile...,https://a0.muscache.com/im/users/58344/profile...,NaN,1,1,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,"Dublin, Dublin City, Ireland",Portobello,Dublin City,NaN,Dublin,Dublin City,Dublin 8,Dublin,"Dublin, Ireland",IE,Ireland,53.331057,-6.269163,f,House,Entire home/apt,3,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,Heating,""Family/kid ...",0.0,$119.00,NaN,NaN,$500.00,$60.00,2,$30.00,20,31,20,20,31,31,20.0,31.0,3 days ago,t,2,2,5,5,2019-01-22,42,16,2011-07-11,2019-01-02,95.0,10.0,9.0,10.0,10.0,10.0,9.0,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.46
1,44077,https://www.airbnb.com/rooms/44077,20190122134607,2019-01-22,cosy comfortable Irish cottage twin,Our house was built in 1937 when there was ple...,"Hi, My name is Teresa and we have 3 rooms ava...",Our house was built in 1937 when there was ple...,none,I like our neighbourhood as there is no shorta...,Dundrum was always known as a Spa Town and the...,There is a bus stop 2 mins up the road from us...,We have a beautiful landscaped garden with tab...,We live on the premises so we are always on ha...,Welcome to our cottage and we hope you will en...,NaN,NaN,https://a0.muscache.com/im/pictures/525706/050...,NaN,193005,https://www.airbnb.com/users/show/193005,Teresa,2010-08-06,Dublin Ireland,"I am a friendly outgoing Irishwoman ,who love...",within an hour,100%,NaN,t,https://a0.muscache.com/im/users/193005/profil...,https://a0.muscache.com/im/users/19

In [87]:
data_extended['host_neighbourhood'].value_counts(dropna=False)

NaN                                    3158
North City Central/O'Connell Street     497
Docklands                               455
Phibsborough                            402
The Liberties                           391
Ranelagh and Rathmines                  284
Ballsbridge                             258
Drumcondra                              244
Old City                                241
Stoneybatter/Arbour Hill                221
Trinity College                         207
Smithfield                              206
Georgian Quarter                        196
Donnybrook                              165
Grafton Street/St. Stephens Green       160
Portobello                              155
Rathgar                                 150
Kilmainham                              142
Temple Bar                              134
Cabra                                    92
Sandymount                               91
East Wall                                84
Clontarf                        

# WHAT ARE THE MOST EXPENSIVE AREAS OF THE CITY TO RENT ON AIRBNB IN DUBLIN?

The aim here is to, for each neighbourhood, break down the rentals into private rooms, single bed apartments and 2 bed apartments. For ech of the 3 categories get an average. The average of these 3 averages will be used to represent the average cost of a rental for that area.

Neighbourhoods get dumped from the statistics if they dont have a minimum number of properties in the 3 categories mentioned above: private rooms, single bedroom house/apt and double bedroom house/apt. This is a bit rough as we are losing the oppurtunity to report on some of those neighbourhoods. Some form of imputing could have been considered when averaging the results if one of the categories was missing.


In [7]:
#create a copy of the data_extended data.
data_areas = data_extended.copy()


In [17]:
def get_list_of_infrequent_values(subset, column, min_freq):
    """ takes a subset of the dataset
    returns a list of the most infrequent values of the column
    """
    v_counts = subset[column].value_counts()
    uncommon_list = list(v_counts[v_counts < min_freq].index)    
    return uncommon_list

In [18]:
#Clean up the data
#1. Remove rows that have NaN for neighbourhood
#2. Remove data from shared rooms as they are fairly infrequent and are likely to have quite different prices
#3. Remove house with 3 or more bedrooms as we will focus on comparing 1 and 2 bedroom rentals (As they are more frequent)
#4. Remove data from neighbourhoods with only a small number of datapoints. We want at least %min_freq% houses and 
#at least %min_freq% private rooms for an area to qualify to be kept.

#1. Remove rows that have NaN for neighbourhood
data_areas.dropna(subset=['host_neighbourhood'], inplace=True)

#2. Remove data from shared rooms
data_areas = data_areas[~(data_areas['room_type'] == 'Shared room')]

#3 Remove houses with 3 or more bedrooms
data_areas = data_areas[~(data_areas['bedrooms'] > 2.0)]

#4. Remove data from neighbourhoods with only a small number of datapoints.

#The min number of times the neighbourhood needs to feature in the list to be considered for comparision 
min_freq = 5

# Get list of neigbourhoods that dont have enough samples for: entire 1 bed home/apt
subset_entirehomes_1b = data_areas[(data_areas['room_type'] =='Entire home/apt') & (data_areas['bedrooms'] ==1.0)]
ignore_hoods1_1bed = get_list_of_infrequent_values(subset_entirehomes_1b, 'host_neighbourhood', min_freq)

# Get list of neigbourhoods that dont have enough samples for: entire 2 bed home/apt
subset_entirehomes_2b = data_areas[(data_areas['room_type'] =='Entire home/apt') & (data_areas['bedrooms'] ==2.0)]
ignore_hoods1_2bed = get_list_of_infrequent_values(subset_entirehomes_2b, 'host_neighbourhood', min_freq)

# Get list of neigbourhoods that dont have enough samples for: private room
subset_privaterooms = data_areas[data_areas['room_type'] =='Private room']
ignore_hoods2 = get_list_of_infrequent_values(subset_privaterooms, 'host_neighbourhood', min_freq)

ignore_hoods = list(set().union(ignore_hoods1_1bed,ignore_hoods1_2bed, ignore_hoods2 ))
print(ignore_hoods)

#get rid of the datapoints that are in the neigbourhoods to be ignored
data_areas_clean = data_areas[~data_areas['host_neighbourhood'].isin(ignore_hoods)].copy()


['Addison Park', 'Beaumont', 'Islandbridge', 'Fairview', 'Tour Eiffel - Champ de Mars', 'Knocklyon', 'Clondalkin', 'Rose Bay/Dover Heights', 'Artane', 'Templeogue', 'Kimmage', 'Ballyfermot', 'Raheny', 'Opéra - Grands Boulevards', 'Ixelles', 'West Village', 'Terézváros - District VI.', 'Santry', 'Finglas', 'Downtown Montreal', 'Rathfarnham', 'Oud-West', 'Edenmore', 'Fulham', 'Isle of Dogs', 'Dartry', 'LB of Haringey', 'Grace Park', 'Churchtown', 'Saint-Germain-des-Prés - Odéon', 'Pelletstown', 'Mount Merrion', 'Ardilea', 'Glasnevin', 'MLK & 183', 'Marino', 'Richmond District', 'Kilmore', 'Terenure', 'Donnycarney', "North City Central/O'Connel Street", 'Dublin Industrial Estate', 'Bonnybrook/Darndale', 'Ballymun', 'Auteuil', 'De Pijp', 'Woluwe-Saint-Lambert']


In [10]:
#Clean up the price column
#1. Get rid of dollar signs and commas
#2. Convert to floats
#3. Investigate outliers

#1. Get rid of dollar signs and commas
#2. Convert to floats
data_areas_clean['price'] = data_areas_clean['price'].replace('[^.0-9]', '', regex=True).astype(float)

#3. Investigate outliers
#Looking at outliers greater than 1000 dollars per night: There are 6 of these. I get rid of all six because..
#Two have broken links. Can't guarantee their integrity
#Three were found to be erroneous prices somehow. 
#One was a correct listings (very fancy rental). However this will skew results in its neighbourhood as it's..
#price is significantly than anything else in Dublin.
#??I should really dig into some of the rentals in the 500 to 1000 dollar branket to investigate.
data_areas_clean = data_areas_clean[~(data_areas_clean['price'] > 1000)]




In [11]:
#Divide the data into Private and Entire house/apt (1 and 2 bedroom) dataframes and sort into groups

data_privateroom = data_areas_clean[data_areas_clean['room_type'] == 'Private room']
privateroom_groups = data_privateroom['price'].groupby(data_areas_clean['host_neighbourhood'])

data_house_1bed = data_areas_clean[(data_areas_clean['room_type'] == 'Entire home/apt') & (data_areas_clean['bedrooms'] == 1.0)]
house_1bed_groups = data_house_1bed['price'].groupby([data_areas_clean['host_neighbourhood']])

data_house_2bed = data_areas_clean[(data_areas_clean['room_type'] == 'Entire home/apt') & (data_areas_clean['bedrooms'] == 2.0)]
house_2bed_groups = data_house_2bed['price'].groupby([data_areas_clean['host_neighbourhood']])



In [12]:
#Combine the averages for the 3 lists

averages = (house_1bed_groups.mean() + house_2bed_groups.mean() + privateroom_groups.mean())/3

#Remove some neighbourhoods that crept in on 2 or fewer lists and therefore have no average across the three

averages = averages.dropna()

averages.sort_values(ascending=False)

host_neighbourhood
Ballsbridge                            138.434360
Docklands                              137.522308
Trinity College                        134.457321
Sandymount                             131.360534
Phibsborough                           126.067187
Georgian Quarter                       124.525343
Grafton Street/St. Stephens Green      123.665263
Drumcondra                             122.935871
Old City                               122.891443
North City Central/O'Connell Street    121.766063
Portobello                             119.925952
Temple Bar                             119.403360
Smithfield                             118.910385
The Liberties                          118.064577
East Wall                              117.465359
Ranelagh and Rathmines                 111.835897
Rathgar                                110.921225
Ringsend/Irishtown                     108.407738
North Strand                           106.368081
Stoneybatter/Arbour Hill       

# WHICH AREAS GET THE BEST AVERAGE LOCATION SCORE?

In [19]:
#create a copy of the data_extended data.
data_popular = data_extended.copy()

In [20]:
data_popular.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,14842,https://www.airbnb.com/rooms/14842,20190122134607,2019-01-22,"Best location Portobello entire house, can sle...","Beautiful entire house, newly revamped, with s...","Gorgeous, newly revamped central Dublin city e...","Beautiful entire house, newly revamped, with s...",none,Portobello is an exceptional place to live - v...,NaN,"There is great public transport - buses, Luas,...",Guests get the whole house to themselves.,I love meeting guests and I give you extensive...,"This house is a charming space, just completel...",NaN,NaN,https://a0.muscache.com/im/pictures/106656330/...,NaN,58344,https://www.airbnb.com/users/show/58344,Hilary,2009-11-29,"Dublin, County Dublin, Ireland",An easy-going and friendly Irish woman. Works ...,within an hour,100%,NaN,t,https://a0.muscache.com/im/users/58344/profile...,https://a0.muscache.com/im/users/58344/profile...,NaN,1,1,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,"Dublin, Dublin City, Ireland",Portobello,Dublin City,NaN,Dublin,Dublin City,Dublin 8,Dublin,"Dublin, Ireland",IE,Ireland,53.331057,-6.269163,f,House,Entire home/apt,3,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,Heating,""Family/kid ...",0.0,$119.00,NaN,NaN,$500.00,$60.00,2,$30.00,20,31,20,20,31,31,20.0,31.0,3 days ago,t,2,2,5,5,2019-01-22,42,16,2011-07-11,2019-01-02,95.0,10.0,9.0,10.0,10.0,10.0,9.0,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.46


In [21]:
#1. Remove rows that have NaN for neighbourhood
data_popular.dropna(subset=['host_neighbourhood'], inplace=True)

#1. Remove rows that have NaN for Location Rating
data_popular.dropna(subset=['review_scores_location'], inplace=True)

In [22]:
data_popular['review_scores_location'].value_counts(dropna=False)

10.0    2902
9.0     1992
8.0      347
7.0       37
6.0       28
4.0        9
2.0        8
5.0        3
Name: review_scores_location, dtype: int64

In [23]:
#Get rid of neighbourhoods that dont have enough samples

#The min number of times the neighbourhood needs to feature in the list to be considered for comparision 
min_freq = 20

ignore_these_areas = get_list_of_infrequent_values(data_popular, 'host_neighbourhood' , min_freq)
print(ignore_these_areas)

#get rid of the datapoints that are in the neigbourhoods to be ignored
data_pop_clean = data_popular[~data_popular['host_neighbourhood'].isin(ignore_these_areas)].copy()

['Bonnybrook/Darndale', 'Dartry', 'Finglas', 'Mount Merrion', 'Addison Park', 'Churchtown', 'Ardilea', 'Knocklyon', 'Marino', 'Ballymun', 'Courtlands', "North City Central/O'Connel Street", 'Ballinteer', 'Templeogue', 'Islandbridge', 'Terézváros - District VI.', 'Riverston Abbey', 'Pelletstown', 'Dublin Industrial Estate', 'Donnycarney', 'Grace Park', 'De Pijp', 'Opéra - Grands Boulevards', 'Edenmore', 'Woluwe-Saint-Lambert', 'Saint-Germain-des-Prés - Odéon', 'MLK & 183', 'Passy', 'Fulham', 'Tour Eiffel - Champ de Mars', 'Richmond District', 'Auteuil', 'West Village', 'Ixelles', 'Downtown Montreal']


In [24]:
rating_groups = data_pop_clean['review_scores_location'].groupby(data_pop_clean['host_neighbourhood'])

In [25]:
rating_groups.mean().sort_values(ascending=False)

host_neighbourhood
Temple Bar                             9.889831
Grafton Street/St. Stephens Green      9.835616
Trinity College                        9.754011
Georgian Quarter                       9.752874
Ballsbridge                            9.709360
Smithfield                             9.696809
Docklands                              9.685864
Old City                               9.678899
Sandymount                             9.666667
Portobello                             9.632353
Dundrum                                9.616667
North City Central/O'Connell Street    9.598624
Ranelagh and Rathmines                 9.584034
Santry                                 9.583333
Booterstown                            9.558824
Rathgar                                9.530973
Clontarf                               9.507692
Monkstown                              9.500000
Raheny                                 9.480000
Blackrock                              9.411765
Ringsend/Irishtown   

# GET AVERAGE COORDINATES FOR EACH NEIGHBOURHOOD TO FACILITATE PLOTING


In [34]:
#create a copy of the data_extended data.
data_coord = data_extended.copy()

In [35]:
#1. Remove rows that have NaN for neighbourhood and coordinates
data_coord.dropna(subset=['host_neighbourhood', 'latitude', 'longitude'], inplace=True)

In [52]:
#Get rid of neighbourhoods that dont have enough samples

#The min number of times the neighbourhood needs to feature in the list to be considered for comparision 
min_freq = 5

ignore_these_areas = get_list_of_infrequent_values(data_coord, 'host_neighbourhood' , min_freq)
print(ignore_these_areas)

#get rid of the datapoints that are in the neigbourhoods to be ignored
data_coord_clean = data_coord[~data_coord['host_neighbourhood'].isin(ignore_these_areas)].copy()

#Remove a couple of specific neighbourhoods (ones that should not be included in the Dublin data)
data_coord_clean = data_coord_clean[~(data_coord_clean['host_neighbourhood'].isin(['Terézváros - District VI.', 'Isle of Dogs']))]

['Donnycarney', 'Grace Park', 'De Pijp', 'West Village', 'Woluwe-Saint-Lambert', 'MLK & 183', 'Saint-Germain-des-Prés - Odéon', 'Ixelles', 'Fulham', 'Downtown Montreal', 'Rose Bay/Dover Heights', 'Opéra - Grands Boulevards', 'Marine Parade', 'LB of Haringey', 'Passy', 'Tour Eiffel - Champ de Mars', 'Richmond District', 'Kilmore', 'Auteuil', 'Oud-West', 'Edenmore', 'French Riviera']


In [53]:
coord_groups = data_coord_clean[['latitude', 'longitude']].groupby(data_coord_clean['host_neighbourhood'])

In [54]:
coord_groups.mean()

,latitude,longitude
host_neighbourhood,,
Addison Park,53.374327,-6.270554
Ardilea,53.299206,-6.226417
Artane,53.380621,-6.209794
Ballinteer,53.276201,-6.250996
Ballsbridge,53.332189,-6.237760
Ballybough,53.366412,-6.250759
Ballyfermot,53.343858,-6.350674
Ballymun,53.397945,-6.272662
Beaumont,53.386831,-6.230951
